In [30]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn import metrics
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

#### Définition de nos variables cibles 

In [19]:
data = pd.read_csv("../data/flight_data.csv")

In [20]:
Y = data['Price']
X = data.drop(['Price'], axis=1)

#### Divisez les données en ensembles d’entraînement et de test

In [21]:
X_train_or, X_test, y_train_or, y_test = train_test_split(X,Y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_or, y_train_or, test_size=0.25)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3,random_state=70)

# Set MLflow

In [22]:
mlflow.set_tracking_uri("sqlite:///mlflowtest.db")
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Flight price prediction tracking")

2023/12/29 20:42:24 INFO mlflow.tracking.fluent: Experiment with name 'Flight price prediction tracking' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/207908173545636087', creation_time=1703878944463, experiment_id='207908173545636087', last_update_time=1703878944463, lifecycle_stage='active', name='Flight price prediction tracking', tags={}>

In [23]:
mlflow.end_run()

In [24]:
mlflow.start_run()
mlflow.log_param("test_size", 0.2)
mlflow.log_param("random_state", 42)

42

# Linear Regression

In [25]:
mlflow.end_run()

In [26]:
with mlflow.start_run(run_name='Linear regression model'):
    reg_model=LinearRegression().fit(X_train,y_train)
    y_pred_val =reg_model.predict(X_val)

    mse_val =mean_squared_error(y_val,y_pred_val)
    mae_val=metrics.mean_absolute_error(y_val,y_pred_val)
    rmse_val=np.sqrt(metrics.mean_absolute_error(y_val,y_pred_val))
    r2_val =r2_score(y_test,y_pred_val)
    
    val_metrics = {'r2_score': r2_val, 'mse':mse_val,'rmse':rmse_val,'mae':mae_val}
    mlflow.log_metrics(val_metrics)
    mlflow.sklearn.log_model(reg_model,"Linear regression model")
mlflow.end_run()
    

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Random Forest

In [59]:
with mlflow.start_run(run_name='Random Forest Regressor Model'):
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_val_rf = rf.predict(X_val)

    mse_val_rf = mean_squared_error(y_val, y_pred_val_rf)
    mae_val_rf = metrics.mean_absolute_error(y_val, y_pred_val_rf)
    rmse_val_rf = np.sqrt(mean_squared_error(y_val, y_pred_val_rf))
    r2_val_rf = r2_score(y_val, y_pred_val_rf)

    val_metrics_rf = {'r2_score': r2_val_rf, 'mse': mse_val_rf, 'rmse': rmse_val_rf, 'mae': mae_val_rf}
    mlflow.log_metrics(val_metrics_rf)
    mlflow.sklearn.log_model(rf, "Random Forest Regressor Model")
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# XG BOOST

In [41]:
with mlflow.start_run(run_name='XGBoost Regressor Model'):
    # Define hyperparameters
    xg_params = {
        'objective': 'reg:squarederror',
        'colsample_bytree': 0.3,
        'learning_rate': 0.1,
        'max_depth': 5,
        'alpha': 10,
        'n_estimators': 10
    }
    xg_reg = xgb.XGBRegressor(**xg_params) 

    xg_reg.fit(X_train, y_train)
    y_pred_val_xgb = xg_reg.predict(X_val)

    mse_val_xgb = mean_squared_error(y_val, y_pred_val_xgb)
    mae_val_xgb = metrics.mean_absolute_error(y_val, y_pred_val_xgb)
    rmse_val_xgb = np.sqrt(mean_squared_error(y_val, y_pred_val_xgb))
    r2_val_xgb = r2_score(y_val, y_pred_val_xgb)

    val_metrics_xgb = {'r2_score': r2_val_xgb, 'mse': mse_val_xgb, 'rmse': rmse_val_xgb, 'mae': mae_val_xgb}
    mlflow.log_metrics(val_metrics_xgb)
    mlflow.log_params(xg_params)
  
    mlflow.xgboost.log_model(xg_reg, "XGBoost Regressor Model")
    
mlflow.end_run()

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\Dell\anaconda3\Lib\site-packages\s

Meilleurs paramètres : {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


# Decision Tree

In [29]:
with mlflow.start_run(run_name='Decision Tree Model'):
    dt_model = DecisionTreeRegressor()

    dt_model.fit(X_val, y_val)
    y_pred_val = dt_model.predict(X_val)


    mse_val =mean_squared_error(y_val,y_pred_val)
    mae_val=metrics.mean_absolute_error(y_val,y_pred_val)
    rmse_val=np.sqrt(metrics.mean_absolute_error(y_val,y_pred_val))
    r2_val =r2_score(y_test,y_pred_val)


    val_metrics = {'r2_score': r2_val, 'mse':mse_val,'rmse':rmse_val,'mae':mae_val}
    mlflow.log_metrics(val_metrics)
    mlflow.sklearn.log_model(dt_model,"Decision TreeRegressor model")
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Performing Hypermater tuning on Random Analysis

In [61]:
test_df = pd.DataFrame({
    "Predicted Price" : rf.predict(X_val),
    "Actual Price" : y_val,
}).reset_index(drop = True)
test_df

,Predicted Price,Actual Price
0,3257.028385,3245.97
1,2541.536270,2869.23
2,1200.434463,797.16
3,7810.489050,6934.20
4,4366.798800,4365.27
...,...,...
453,6424.991300,7589.40
454,4123.583100,3985.80
455,1336.034700,1228.50
456,11076.565500,10622.43


In [62]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Définition du paramètre grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Création de l'objet GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, scoring='r2', cv=5, verbose=1)

# Ajustement du modèle aux données
grid_search.fit(X_train, y_train)

# Obtention des meilleurs paramètres
best_params = grid_search.best_params_

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :", best_params)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\Dell\anaconda3\Lib\site-packages\s

Meilleurs paramètres : {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [63]:
with mlflow.start_run(run_name='Random Forest Regressor Model'):
    rf_params={'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
    rf = RandomForestRegressor(**rf_params)
    rf.fit(X_train, y_train)
    y_pred_val_rf = rf.predict(X_val)

    mse_val_rf = mean_squared_error(y_val, y_pred_val_rf)
    mae_val_rf = metrics.mean_absolute_error(y_val, y_pred_val_rf)
    rmse_val_rf = np.sqrt(mean_squared_error(y_val, y_pred_val_rf))
    r2_val_rf = r2_score(y_val, y_pred_val_rf)

    val_metrics_rf = {'r2_score': r2_val_rf, 'mse': mse_val_rf, 'rmse': rmse_val_rf, 'mae': mae_val_rf}
    mlflow.log_metrics(val_metrics_rf)
    mlflow.sklearn.log_model(rf, "Random Forest Regressor Model")
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [64]:
test_df = pd.DataFrame({
    "Predicted Price" : rf.predict(X_val),
    "Actual Price" : y_val,
}).reset_index(drop = True)
test_df

,Predicted Price,Actual Price
0,3364.780055,3245.97
1,2801.744400,2869.23
2,1191.346975,797.16
3,7226.928800,6934.20
4,4465.429150,4365.27
...,...,...
453,6094.464740,7589.40
454,4366.896625,3985.80
455,1390.443600,1228.50
456,10880.469600,10622.43


##### Hyperparameter tuning didn't improve the model accuracy